Problem Statement
---------------------------
You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships to give accurate estimates of how many crew members a ship will require.

[Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

### Create Spark Session

In [1]:
from pyspark.sql import SparkSession

In [2]:
sessn = SparkSession.builder.appName('PySparkProj').getOrCreate()

In [3]:
sessn

### Read csv File and display information about Dataset

In [4]:
data = sessn.read.csv(path='cruise_ship_info.csv',header=True,inferSchema=True)

In [6]:
data.count()

158

In [7]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [8]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [9]:
data.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|      NaN|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [78]:
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



#### CONVERTING THE STRING COLUMN 'Cruise_line' TO STRING INDEXER

In [18]:
from pyspark.ml.feature import StringIndexer

In [23]:
indexer = StringIndexer(inputCol='Cruise_line',outputCol='Cruise_line_index')

In [27]:
indexed_data = indexer.fit(data).transform(data)

In [33]:
indexed_data.show(40)

+-------------+-----------+---+------------------+----------+------+------+-----------------+-----+-----------------+
|    Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density| crew|Cruise_line_index|
+-------------+-----------+---+------------------+----------+------+------+-----------------+-----+-----------------+
|      Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64| 3.55|             16.0|
|        Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64| 3.55|             16.0|
|  Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8|  6.7|              1.0|
|     Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99| 19.1|              1.0|
|      Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36| 10.0|              1.0|
|      Ecstasy|   Carnival| 22|            70.367|     2

### CONVERT csv TO MlLib FORMAT

In [38]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [39]:
indexed_data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_index']

In [40]:
assembler = VectorAssembler(inputCols=['Cruise_line_index','Age','Tonnage','passengers','length','cabins','passenger_density'],
                            outputCol='features')

In [41]:
mllib_data = assembler.transform(indexed_data)

In [42]:
mllib_data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_line_index: double (nullable = false)
 |-- features: vector (nullable = true)



In [43]:
final_mllib_data = mllib_data.select('features','crew')

In [45]:
final_mllib_data.show(5)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[16.0,6.0,30.2769...|3.55|
|[16.0,6.0,30.2769...|3.55|
|[1.0,26.0,47.262,...| 6.7|
|[1.0,11.0,110.0,2...|19.1|
|[1.0,17.0,101.353...|10.0|
+--------------------+----+
only showing top 5 rows



In [46]:
final_mllib_data.head(1)

[Row(features=DenseVector([16.0, 6.0, 30.277, 6.94, 5.94, 3.55, 42.64]), crew=3.55)]

### Train Test Split

In [50]:
train_data,test_data = final_mllib_data.randomSplit([0.7,0.3])

In [51]:
train_data.count()

111

In [52]:
test_data.count()

47

In [53]:
final_mllib_data.count()

158

In [54]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               111|
|   mean|  7.78513513513514|
| stddev|3.5879223830828657|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [55]:
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               47|
|   mean|7.815531914893618|
| stddev|3.332979349073528|
|    min|             0.88|
|    max|             13.6|
+-------+-----------------+



In [56]:
final_mllib_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



### Build Linear Regression Model

In [103]:
from pyspark.ml.regression import LinearRegression

In [57]:
lr_ob = LinearRegression(featuresCol='features',labelCol='crew')

In [58]:
lrm = lr_ob.fit(train_data)

In [62]:
summ = lrm.summary

In [63]:
summ.meanAbsoluteError

0.6114927837760632

In [64]:
summ.meanSquaredError

1.0492984706281012

In [65]:
summ.r2

0.9177486083740723

In [66]:
summ.rootMeanSquaredError

1.0243527081177173

### Evaluate the Linear Regression Model

In [67]:
pred = lrm.evaluate(test_data)

In [68]:
pred.meanAbsoluteError

0.6282592291613499

In [69]:
pred.meanSquaredError

0.5576229709091496

In [104]:
pred.predictionCol

'prediction'

In [105]:
pred.rootMeanSquaredError

0.746741569024485

In [106]:
pred.r2

0.9487120386531305

In [109]:
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               47|
|   mean|7.815531914893618|
| stddev|3.332979349073528|
|    min|             0.88|
|    max|             13.6|
+-------+-----------------+



#### The performance of the Model is quite good as visible from the RMSE and comparing it with the stats of the test data

In [73]:
pred.predictions.show()

+--------------------+-----+------------------+
|            features| crew|        prediction|
+--------------------+-----+------------------+
|[0.0,5.0,160.0,36...| 13.6|15.227733564109487|
|[0.0,9.0,90.09,25...| 8.69|  9.20929623207827|
|[0.0,10.0,138.0,3...|11.85|13.105090685569316|
|[0.0,11.0,90.09,2...| 8.48| 8.784672554742864|
|[0.0,12.0,90.09,2...| 8.68|  8.77451794165236|
|[0.0,13.0,138.0,3...|11.76|13.074626846297809|
|[0.0,21.0,73.941,...| 8.22| 9.017217356340105|
|[0.0,25.0,73.192,...| 8.08| 8.456099648686092|
|[1.0,9.0,88.5,21....| 10.3| 9.562599190631436|
|[1.0,10.0,110.0,2...| 11.6|12.252921268638797|
|[1.0,12.0,88.5,21...|10.29| 9.477001562566148|
|[1.0,15.0,70.367,...|  9.2| 8.702511705300093|
|[1.0,17.0,101.353...| 10.0| 10.90852113235615|
|[1.0,18.0,70.367,...|  9.2| 8.672047866028587|
|[1.0,22.0,70.367,...|  9.2| 8.631429413666577|
|[2.0,6.0,113.0,37...| 12.0|11.722506759922124|
|[2.0,7.0,116.0,31...| 12.0|12.836537263334375|
|[2.0,9.0,113.0,26...|12.38|11.448967848

In [75]:
pred.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -1.6277335641094872|
| -0.5192962320782701|
|  -1.255090685569316|
|-0.30467255474286326|
|-0.09451794165236116|
| -1.3146268462978092|
| -0.7972173563401039|
|-0.37609964868609147|
|  0.7374008093685642|
| -0.6529212686387975|
|  0.8129984374338513|
| 0.49748829469990596|
| -0.9085211323561495|
|  0.5279521339714126|
|  0.5685705863334221|
| 0.27749324007787557|
| -0.8365372633343746|
|  0.9310321512501751|
|  0.4560555324676159|
| -1.4098765979858667|
+--------------------+
only showing top 20 rows



### FIND CORRELATION BETWEEN FEATURES

In [110]:
from pyspark.sql.functions import corr

In [111]:
data.select(corr('crew','passenger_density')).show()

+-----------------------------+
|corr(crew, passenger_density)|
+-----------------------------+
|         -0.15550928421699717|
+-----------------------------+



In [112]:
data.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [113]:
data.select(corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+



In [86]:
#

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]